In [1]:
import os
import random

import torch
from torch import nn
from torchvision import datasets, models
from torchvision.transforms import v2
from torch.utils.data import DataLoader

from PIL import Image, ImageOps

from tqdm import tqdm

In [2]:
class LetterboxPad:
    """
    Pads images with black borders to make them square.
    This prevents distortion when resizing.
    """
    def __call__(self, img):
        w, h = img.size
        max_dim = max(w, h)
        padding = ImageOps.pad(img, (max_dim, max_dim), color="black", centering=(0.5, 0.5))
        return padding

augment_transform= v2.Compose([
    LetterboxPad(),
    v2.Lambda(lambda img: img.convert("RGB")),
    v2.Resize(232),
    v2.CenterCrop(224),
    v2.RandomHorizontalFlip(0.2),
    # v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    v2.RandomGrayscale(0.2),
    v2.RandomRotation(10)
])

In [3]:
dataset_root= r"D:\ASL_Project\dataset"
augmented_dataset_root= r"D:\ASL_Project\augmented_dataset2"

num_augmentations= 2
os.makedirs(augmented_dataset_root, exist_ok= True)
classes= ["approved", "rejected"]

for cls in classes:
    input_dir= os.path.join(dataset_root, cls)
    output_dir= os.path.join(augmented_dataset_root, cls)
    os.makedirs(output_dir, exist_ok= True)
    
    image_files= os.listdir(input_dir)
    
    for img_name in tqdm(image_files, desc=f"processing the {cls} folder"):
        img_path= os.path.join(input_dir, img_name)
        
        try:
            image= Image.open(img_path).convert("RGB")
        except:
            continue
        
        base_name= os.path.splitext(img_name)[0]
        
        processed_original= v2.Compose([
            v2.Resize(232),
            v2.CenterCrop(224)
        ])(image)
        
        processed_original.save(os.path.join(output_dir, f"{base_name}_original.jpg"))
        
        for i in range(num_augmentations):
            augmented_image= augment_transform(image)
            save_name=f"{base_name}_augmented{i}.jpg"
            augmented_image.save(os.path.join(output_dir, save_name))

processing the approved folder:   0%|          | 0/559 [00:00<?, ?it/s]

processing the rejected folder: 100%|██████████| 207/207 [00:23<00:00,  8.67it/s]
